In this notebook we use NFA for text calssification. it works on the proncipal of linear algebra.

In [0]:
import os
import pandas as pd

In [0]:
dataframe = pd.read_csv('quora_questions.csv', nrows=200000)
dataframe.columns = ["questions"]

In [141]:
dataframe.head()

,questions
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer# to fit the NMF we use tfid

TFIDF measure the document according to how important a word to document in a coolection. it can also filtered out stop words. it works by multiplying the two metrics

In [0]:
tfidf = TfidfVectorizer(max_df=1.0, min_df=1, stop_words="english")#in this model we use the default values of min max

In [0]:
dtm_with_tfidf = tfidf.fit_transform(dataframe["questions"])#as same we do it in LDA

In [145]:
dtm_with_tfidf

<200000x50470 sparse matrix of type '<class 'numpy.float64'>'
	with 1004332 stored elements in Compressed Sparse Row format>

In [0]:
from sklearn.decomposition import NMF #IMPORT the model from sklearn

In [0]:
nmf_model = NMF(n_components=15, random_state=1)#we added 15 topic components this time

In [148]:
nmf_model.fit(dtm_with_tfidf)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=15, random_state=1, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [149]:
word_list = []
probability_list = []

top_number = 20
count = 0
for probability_number in nmf_model.components_: # model.components contains the prob of each word for each doc
    text_message = f"Top words for topic {count} are : "
    print(text_message)    
    for number in probability_number.argsort()[-top_number:]: # we're only interested in the top words
        print([tfidf.get_feature_names()[number]], end= "")
        word_list.append([tfidf.get_feature_names()[number]])
        probability_list.append(number)
    #show_chart(word_list, probability_list, text_message)
    print("\n")  
    count += 1

Top words for topic 0 are : 
['hollywood']['read']['site']['thing']['place']['website']['visit']['time']['ways']['places']['phone']['buy']['laptop']['movie']['2016']['books']['book']['movies']['way']['best']

Top words for topic 1 are : 
['need']['says']['universities']['use']['grads']['majors']['differ']['recruit']['looking']['sex']['really']['exist']['compare']['cost']['time']['long']['feel']['work']['mean']['does']

Top words for topic 2 are : 
['users']['search']['marked']['follow']['use']['add']['answered']['post']['needing']['easily']['improvement']['delete']['asked']['google']['answers']['answer']['ask']['question']['questions']['quora']

Top words for topic 3 are : 
['website']['using']['facebook']['friends']['making']['investment']['job']['internet']['black']['free']['easiest']['easy']['home']['youtube']['ways']['way']['earn']['online']['make']['money']

Top words for topic 4 are : 
['happy']['decision']['planets']['balance']['things']['did']['death']['earth']['live']['changed

From the above output we can see that topic 14 related to Medicine and Healthcare. topic 12 related to Literature, Languages, and Communication 11 is related to Goverment politics. now we see these word lists are related to each other so we add the relevant topic number into the original dataframe.

In [0]:
textfile_topics = nmf_model.transform(dtm_with_tfidf)

In [0]:

topic_list = []
# Textfile_topics is a list of arrays containing 
# all index positions of words for each textfile
for popular_index_pos in textfile_topics:
    # Get the max index position in each array
    # and add to the topic_list list
    topic_list.append(popular_index_pos.argmax())

# Add a new column to the dataframe
dataframe["Topic number"] = topic_list

In [152]:
dataframe

,questions,Topic number
0,What is the step by step guide to invest in sh...,8
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,13
2,How can I increase the speed of my internet co...,3
3,Why am I mentally very lonely? How can I solve...,11
4,"Which one dissolve in water quikly sugar, salt...",14
...,...,...
199995,Why was the Battle of Vimy Ridge so important?,4
199996,Which of these TV shows should I watch next?,0
199997,Should I change my name?,4
199998,Should I buy the new MacBook 2016 or one from ...,13


In [0]:
topic_list = {0: "News, Entertainment, and Pop Culture", 
              1: "horoscopes", 
              2: "Life, Relationships, and Self", 
              3: "Education, Schools, and Learning", 
              4: "Recreation, Sports, Travel, and Activities", 
              5: "Humanities", 
              6: "Science, Technology, Engineering, and Mathematics", 
              7: "Business, Work, and Careers", 
              8: "Art, Design, and Style", 
              9: "Honours and Recognition", 
              10: "Food, Cuisines, and Cooking", 
              11: "Politics, Law, Government, and Judiciary", 
              12: "Literature, Languages, and Communication", 
              13: "Major Concepts", 
              14: "Medicine and Healthcare"}



In [0]:
topic_no_to_topic = dataframe["Topic number"].map(topic_list)

In [0]:
dataframe["Topic desc"] = topic_no_to_topic


In [156]:

dataframe.head(20)

,questions,Topic number,Topic desc
0,What is the step by step guide to invest in sh...,8,"Art, Design, and Style"
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,13,Major Concepts
2,How can I increase the speed of my internet co...,3,"Education, Schools, and Learning"
3,Why am I mentally very lonely? How can I solve...,11,"Politics, Law, Government, and Judiciary"
4,"Which one dissolve in water quikly sugar, salt...",14,Medicine and Healthcare
5,Astrology: I am a Capricorn Sun Cap moon and c...,1,horoscopes
6,Should I buy tiago?,0,"News, Entertainment, and Pop Culture"
7,How can I be a good geologist?,10,"Food, Cuisines, and Cooking"
8,When do you use ã‚· instead of ã—?,7,"Business, Work, and Careers"
9,Motorola (company): Can I hack my Charter Moto...,8,"Art, Design, and Style"


In [0]:
dataframe.to_csv('quora_supervised.csv')#transform the result into quora_supervised dataset to use it into another question.

References:

